In [2]:
import csv
from neo4j import GraphDatabase

# 1. กำหนดค่าการเชื่อมต่อ Neo4j ของคุณ
URI = "bolt://localhost:7687"  # เปลี่ยนหากพอร์ตไม่ตรง
USER = "neo4j"
PASSWORD = "12345678" # ***กรุณาเปลี่ยนรหัสผ่าน***
COMPANIES_CSV_FILE = "data/companies.csv" # ชื่อไฟล์ CSV

# 2. เชื่อมต่อฐานข้อมูล
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

# 3. ฟังก์ชันสำหรับสร้าง Node Company
# ใช้ Transaction เพื่อให้การทำงานปลอดภัย
def create_company_node(tx, company_id, name):
    # คำสั่ง Cypher ใช้ $ เพื่อรับ Parameter จาก Python
    query = """
    CREATE (c:Company {companyId: $id, name: $name})
    """
    tx.run(query, id=company_id, name=name)

# 4. ฟังก์ชันหลักสำหรับอ่านและ Import ข้อมูล
def import_companies():
    print(f"กำลังเริ่ม Import ไฟล์: {COMPANIES_CSV_FILE}...")
    
    # ใช้ csv.DictReader เพื่ออ่านข้อมูลจากไฟล์ companies.csv
    # ตรวจสอบให้แน่ใจว่าไฟล์นี้อยู่ในโฟลเดอร์เดียวกับ script Python
    with open(COMPANIES_CSV_FILE, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        
        # ใช้ session เพื่อจัดการการสื่อสารกับฐานข้อมูล
        with driver.session() as session:
            count = 0
            for row in reader:
                # เรียกใช้ฟังก์ชัน create_company_node ใน Transaction (write_transaction)
                session.write_transaction(
                    create_company_node, 
                    row['companyId:ID'], # ต้องใช้ชื่อคอลัมน์จาก CSV เป็น key
                    row['name:STRING']
                )
                count += 1
            print(f"✅ Import Node Company เสร็จสิ้น: {count} รายการ")

# 5. รันโปรแกรม
if __name__ == "__main__":
    try:
        import_companies()
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อหรือ Import: {e}")
    finally:
        driver.close() # ปิดการเชื่อมต่อเมื่อทำงานเสร็จ

กำลังเริ่ม Import ไฟล์: data/companies.csv...
เกิดข้อผิดพลาดในการเชื่อมต่อหรือ Import: 'Session' object has no attribute 'write_transaction'


In [5]:
from neo4j import GraphDatabase

# --- ข้อมูล Mock Data ในรูปแบบ Python Dictionary ---
# (นี่คือตัวอย่าง subset จาก CSV ที่ให้ไปก่อนหน้านี้)

COMPANIES = [
    {'id': 'C001', 'name': 'Advanced Info Service PCL'},
    {'id': 'C002', 'name': 'Delta Electronics (Thailand) PCL'},
    {'id': 'C003', 'name': 'Kasikornbank PCL'}
]

STOCKS = [
    {'ticker': 'ADVANC', 'name': 'Advanced Info Service'},
    {'ticker': 'DELTA', 'name': 'Delta Electronics'},
    {'ticker': 'KBANK', 'name': 'Kasikornbank'}
]

HAS_STOCK_RELS = [
    {'comp_id': 'C001', 'stock_id': 'ADVANC'},
    {'comp_id': 'C002', 'stock_id': 'DELTA'},
    {'comp_id': 'C003', 'stock_id': 'KBANK'}
]

# --- Class สำหรับจัดการฐานข้อมูล ---
class Neo4jImporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    # 1. ฟังก์ชันสร้างโหนด
    def create_nodes(self, data, node_label, id_property, name_property):
        # UNWIND Cypher ใช้สำหรับรับ List ของ Parameters (data)
        query = f"""
        UNWIND $data AS row
        MERGE (n:{node_label} {{{id_property}: row.{id_property}}})
        SET n.name = row.{name_property}
        """
        
        with self.driver.session() as session:
            session.run(query, data=data)
            print(f"✅ Created/Merged {len(data)} {node_label} nodes.")

    # 2. ฟังก์ชันสร้างความสัมพันธ์
    def create_relationships(self, data):
        query = """
        UNWIND $data AS rel
        MATCH (c:Company {companyId: rel.comp_id})
        MATCH (s:Stock {ticker: rel.stock_id})
        MERGE (c)-[:HAS_STOCK]->(s)
        """
        
        with self.driver.session() as session:
            session.run(query, data=data)
            print(f"✅ Created/Merged {len(data)} HAS_STOCK relationships.")


if __name__ == "__main__":
    URI = "neo4j://127.0.0.1:7687"
    USER = "neo4j"
    PASSWORD = "12345678"

    importer = Neo4jImporter(URI, USER, PASSWORD)
    
    print("--- เริ่ม Import ข้อมูล Thai Stock Mock Data ---")
    
    # 1. สร้างโหนด Company
    importer.create_nodes(
        data=COMPANIES,
        node_label="Company",
        id_property="companyId",
        name_property="name"
    )
    
    # 2. สร้างโหนด Stock
    importer.create_nodes(
        data=STOCKS,
        node_label="Stock",
        id_property="ticker", # ใช้ 'ticker' เป็น ID
        name_property="name"
    )

    # 3. สร้างความสัมพันธ์ HAS_STOCK
    importer.create_relationships(HAS_STOCK_RELS)

    importer.close()
    print("--- Import เสร็จสมบูรณ์ ---")

--- เริ่ม Import ข้อมูล Thai Stock Mock Data ---


ClientError: {neo4j_code: Neo.ClientError.Statement.SemanticError} {message: Cannot merge the following node because of null property value for 'companyId': (:Company {companyId: null})} {gql_status: 22G03} {gql_status_description: error: data exception - invalid value type}

In [9]:
URI = "neo4j://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "12345678"

importer = Neo4jImporter(URI, USER, PASSWORD)

print("--- เริ่ม Import ข้อมูล Thai Stock Mock Data ---")

NameError: name 'Neo4jImporter' is not defined

In [23]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "12345678"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [24]:
print(driver.verify_connectivity())

None


In [25]:
# Example data for nodes
nodes_data = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25},
    {"name": "Charlie", "age": 35}
]

# Example data for relationships
relationships_data = [
    {"from": "Alice", "to": "Bob", "type": "FRIENDS_WITH"},
    {"from": "Bob", "to": "Charlie", "type": "KNOWS"}
]

In [26]:
# Cypher query to create Person nodes
create_nodes_query = """
UNWIND $data AS person
MERGE (p:Person {name: person.name})
SET p.age = person.age
"""

# Cypher query to create relationships
create_relationships_query = """
UNWIND $data AS rel
MATCH (p1:Person {name: rel.from})
MATCH (p2:Person {name: rel.to})
MERGE (p1)-[:HAS_RELATIONSHIP {type: rel.type}]->(p2)
"""

In [29]:
driver.session()

In [30]:
def import_data(tx, query, data):
    tx.run(query, data=data)

with driver.session() as session:
    session.write_transaction(import_data, create_nodes_query, nodes_data)
    session.write_transaction(import_data, create_relationships_query, relationships_data)

driver.close()

AttributeError: 'Session' object has no attribute 'write_transaction'

In [31]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [34]:
summary = driver.execute_query("""
    CREATE (a:Person {name: $name})
    CREATE (b:Person {name: $friendName})
    CREATE (a)-[:KNOWS]->(b)
    """,
    name="Alice", friendName="David",
    database_="test",
).summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

DriverError: Driver closed

In [2]:
import pandas as pd

# ข้อมูลตัวอย่าง: หุ้นไทย
data = {
    'symbol': ['PTT', 'AOT', 'KBANK', 'SCB', 'CPALL'],
    'name': ['PTT Public Company', 'Airports of Thailand', 'Kasikornbank', 'SCB X', 'CP All'],
    'sector': ['Energy', 'Transportation', 'Banking', 'Banking', 'Commerce'],
    'price': [35.0, 70.0, 130.0, 105.0, 60.0]
}

df = pd.DataFrame(data)
print(df)

  symbol                  name          sector  price
0    PTT    PTT Public Company          Energy   35.0
1    AOT  Airports of Thailand  Transportation   70.0
2  KBANK          Kasikornbank         Banking  130.0
3    SCB                 SCB X         Banking  105.0
4  CPALL                CP All        Commerce   60.0


In [3]:
from neo4j import GraphDatabase
import pandas as pd

# 1. การตั้งค่าการเชื่อมต่อ (แก้ password ให้ตรงกับที่คุณตั้ง)
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")  # <--- ใส่ password ของคุณที่นี่

# 2. ฟังก์ชันสำหรับสร้างข้อมูลใน Neo4j
def create_stock_data(tx, symbol, name, sector, price):
    query = """
    MERGE (c:Company {symbol: $symbol})
    SET c.name = $name, c.price = $price
    
    MERGE (s:Sector {name: $sector})
    
    MERGE (c)-[:IN_SECTOR]->(s)
    """
    # MERGE คือคำสั่งที่บอกว่า "ถ้ามีอยู่แล้วไม่ต้องสร้างใหม่ ถ้ายังไม่มีให้สร้าง"
    # ช่วยป้องกันข้อมูลซ้ำซ้อน
    tx.run(query, symbol=symbol, name=name, sector=sector, price=price)

# 3. เตรียมข้อมูล (ใช้ DataFrame จากข้อ 4)
data = {
    'symbol': ['PTT', 'AOT', 'KBANK', 'SCB', 'CPALL'],
    'name': ['PTT Public Company', 'Airports of Thailand', 'Kasikornbank', 'SCB X', 'CP All'],
    'sector': ['Energy', 'Transportation', 'Banking', 'Banking', 'Commerce'],
    'price': [35.0, 70.0, 130.0, 105.0, 60.0]
}
df = pd.DataFrame(data)

# 4. เชื่อมต่อและรันการ Import
# ใช้ with driver เพื่อจัดการ connection ให้อัตโนมัติ
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        print("กำลังเริ่มนำเข้าข้อมูล...")
        
        # วนลูปข้อมูลใน DataFrame ทีละแถว
        for index, row in df.iterrows():
            session.execute_write(
                create_stock_data, 
                row['symbol'], 
                row['name'], 
                row['sector'], 
                row['price']
            )
            print(f"Imported: {row['symbol']}")
            
        print("เสร็จสิ้นการนำเข้าข้อมูล!")

กำลังเริ่มนำเข้าข้อมูล...
Imported: PTT
Imported: AOT
Imported: KBANK
Imported: SCB
Imported: CPALL
เสร็จสิ้นการนำเข้าข้อมูล!


In [5]:
def create_stocks_batch(tx, data_list):
    query = """
    UNWIND $data AS row
    MERGE (c:Company {symbol: row.symbol})
    SET c.name = row.name, c.price = row.price
    
    MERGE (s:Sector {name: row.sector})
    
    MERGE (c)-[:IN_SECTOR]->(s)
    """
    tx.run(query, data=data_list)

# แปลง DataFrame เป็น List of Dictionaries
data_list = df.to_dict('records')

# รันทีเดียว
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        session.execute_write(create_stocks_batch, data_list)

In [6]:
data_list

[{'symbol': 'PTT',
  'name': 'PTT Public Company',
  'sector': 'Energy',
  'price': 35.0},
 {'symbol': 'AOT',
  'name': 'Airports of Thailand',
  'sector': 'Transportation',
  'price': 70.0},
 {'symbol': 'KBANK',
  'name': 'Kasikornbank',
  'sector': 'Banking',
  'price': 130.0},
 {'symbol': 'SCB', 'name': 'SCB X', 'sector': 'Banking', 'price': 105.0},
 {'symbol': 'CPALL', 'name': 'CP All', 'sector': 'Commerce', 'price': 60.0}]

In [7]:
import pandas as pd

# ข้อมูลความสัมพันธ์ระหว่างบริษัท
# Source = บริษัทต้นทาง, Target = บริษัทปลายทาง, Rel_Type = ชนิดความสัมพันธ์, Stake = สัดส่วนการถือหุ้น (ถ้ามี)
rel_data = {
    'source': ['SCB', 'PTT', 'CPALL'],
    'target': ['KBANK', 'AOT', 'MAKRO'], # สมมติ MAKRO เพิ่มเข้ามา
    'rel_type': ['PARTNER_WITH', 'SUPPLIES_TO', 'OWNS'],
    'percent': [None, None, 60.0] # ตัวอย่าง: CPALL ถือหุ้น MAKRO 60%
}

# หมายเหตุ: ต้องมั่นใจว่าบริษัทใน 'source' และ 'target' มี Node อยู่ในระบบแล้ว 
# (หรือเราจะเขียนโค้ดให้สร้าง Node อัตโนมัติถ้ายังไม่มีก็ได้)

df_rel = pd.DataFrame(rel_data)
print(df_rel)

  source target      rel_type  percent
0    SCB  KBANK  PARTNER_WITH      NaN
1    PTT    AOT   SUPPLIES_TO      NaN
2  CPALL  MAKRO          OWNS     60.0


In [9]:
from neo4j import GraphDatabase

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "12345678") # อย่าลืมแก้ password

def create_company_relationships(tx, data_list):
    query = """
    UNWIND $data AS row
    
    // 1. หาหรือสร้าง Node ต้นทาง (Source)
    MERGE (src:Company {symbol: row.source})
    
    // 2. หาหรือสร้าง Node ปลายทาง (Target)
    MERGE (tgt:Company {symbol: row.target})
    
    // 3. สร้างความสัมพันธ์ตามประเภท (ใช้ Case หรือแยก Query ก็ได้ แต่วิธีนี้ Dynamic กว่า)
    // ใช้ APOC library จะง่ายกว่าถ้า type เปลี่ยนไปเรื่อยๆ แต่ถ้า pure Cypher แยกเคสจะชัวร์กว่า
    // ในที่นี้ขอยกตัวอย่างแยก Case เพื่อความเข้าใจง่าย
    
    WITH src, tgt, row
    CALL apoc.do.case([
        row.rel_type = 'OWNS', 
            'MERGE (src)-[r:OWNS]->(tgt) SET r.percent = row.percent RETURN r',
        row.rel_type = 'PARTNER_WITH', 
            'MERGE (src)-[r:PARTNER_WITH]->(tgt) RETURN r',
        row.rel_type = 'SUPPLIES_TO', 
            'MERGE (src)-[r:SUPPLIES_TO]->(tgt) RETURN r'
    ], '', {src:src, tgt:tgt, row:row}) YIELD value
    RETURN value
    """
    
    # ** แบบง่าย (ไม่ต้องใช้ APOC) **
    # ถ้าคุณไม่ได้ลง APOC Plugin ให้ใช้ Query แยกแบบด้านล่างนี้แทน:
    
    query_simple = """
    UNWIND $data AS row
    MATCH (src:Company {symbol: row.source})
    MATCH (tgt:Company {symbol: row.target})
    
    FOREACH (_ IN CASE WHEN row.rel_type = 'OWNS' THEN [1] ELSE [] END |
        MERGE (src)-[r:OWNS]->(tgt) SET r.percent = row.percent
    )
    FOREACH (_ IN CASE WHEN row.rel_type = 'PARTNER_WITH' THEN [1] ELSE [] END |
        MERGE (src)-[:PARTNER_WITH]->(tgt)
    )
    FOREACH (_ IN CASE WHEN row.rel_type = 'SUPPLIES_TO' THEN [1] ELSE [] END |
        MERGE (src)-[:SUPPLIES_TO]->(tgt)
    )
    """
    
    tx.run(query_simple, data=data_list)

# แปลง Data เป็น List of Dicts
rel_list = df_rel.to_dict('records')

# รันคำสั่ง
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        # อย่าลืมสร้าง Node MAKRO ก่อน หรือรัน create_stock_data ให้ครบทุกตัวก่อนจะดีที่สุด
        # แต่ใน query_simple ผมใช้ MATCH คือต้องมี Node อยู่แล้วถึงจะสร้างเส้นได้
        
        # (Optional) สร้าง Node MAKRO ดักไว้ก่อนเพื่อกัน error
        session.run("MERGE (:Company {symbol: 'MAKRO', name: 'Siam Makro'})") 
        
        print("กำลังสร้างความสัมพันธ์...")
        session.execute_write(create_company_relationships, rel_list)
        print("สร้างความสัมพันธ์เรียบร้อย!")

กำลังสร้างความสัมพันธ์...
สร้างความสัมพันธ์เรียบร้อย!
